In [1]:
import torch
from torchvision.io.video import read_video
from torchvision.models.optical_flow import raft_large, Raft_Large_Weights, raft_small, Raft_Small_Weights
from torchvision.transforms.functional import resize
import torchvision.transforms as T
from torchvision.utils import flow_to_image
from torch.utils.data import Dataset, DataLoader
# from torch.cuda.amp import autocast
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import numpy as np
import av

### Display video file

In [23]:
dataset_path = Path('H:/Datasets/Celeb_DF/YouTube-real')
video_path = "00000.mp4"
data_path = dataset_path / video_path
# data_path = "H:\Datasets\Celeb_DF\YouTube-real\00273.mp4"

print(data_path)
# Load the video using torchvision
vid, _, _ = read_video(data_path, output_format="TCHW", pts_unit='sec')  # TCHW: Time, Channels, Height, Width
# vid = vid[:32]  # Optionally, shorten the duration if needed (e.g., first 32 frames)
print(vid.shape)

# Display the optical flow image using OpenCV
for frame in vid:
    fr = cv2.cvtColor(frame.numpy().transpose(1, 2, 0), cv2.COLOR_BGR2RGB)
    # print(fr.shape) # 500,892,3
    cv2.imshow('input', fr)
    # Wait for 25ms before moving to the next frame
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break  # Exit the loop if 'q' is pressed

# Release the video window
cv2.destroyAllWindows()

H:\Datasets\Celeb_DF\YouTube-real\00000.mp4
torch.Size([450, 3, 500, 892])


### Load Raft model

In [2]:
# Load the RAFT model with pre-trained weights
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = raft_large(weights=Raft_Large_Weights.DEFAULT).to(device)
model.eval()

RAFT(
  (feature_encoder): FeatureEncoder(
    (convnormrelu): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): ReLU(inplace=True)
    )
    (layer1): Sequential(
      (0): ResidualBlock(
        (convnormrelu1): Conv2dNormActivation(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (2): ReLU(inplace=True)
        )
        (convnormrelu2): Conv2dNormActivation(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (2): ReLU(inplace=True)
        )
        (downsample): Identity()
        (relu): ReLU(inplace=True)
      )
      (1): ResidualBlock(
        (

In [8]:
# # Load the RAFT model with pre-trained weights
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = raft_small(weights=Raft_Small_Weights.DEFAULT).to(device)
# model.eval()

RAFT(
  (feature_encoder): FeatureEncoder(
    (convnormrelu): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): ReLU(inplace=True)
    )
    (layer1): Sequential(
      (0): BottleneckBlock(
        (convnormrelu1): Conv2dNormActivation(
          (0): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (1): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (2): ReLU(inplace=True)
        )
        (convnormrelu2): Conv2dNormActivation(
          (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (2): ReLU(inplace=True)
        )
        (convnormrelu3): Conv2dNormActivation(
          (0): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (1): Inst

### Process Videos

In [3]:
# # Preprocessing function (based on RAFT's input expectations)
# def preprocess_frame(frame, device):
#     frame_tensor = frame.float() / 255.0  # Normalize to [0, 1]
#     frame_resized = resize(frame_tensor, size=[496, 888])  # Resize to match RAFT expectations 500, 892 ->  496, 888
#     return frame_resized.unsqueeze(0).to(device)  # Add batch dimension and move to device

In [5]:
# dataset_path = Path('H:/Datasets/Celeb_DF/YouTube-real')
# video_path = "00000.mp4"
# data_path = dataset_path / video_path
# # data_path = "H:\Datasets\Celeb_DF\YouTube-real\00273.mp4"

# # Load the video using torchvision
# vid, _, _ = read_video(data_path, output_format="TCHW", pts_unit='sec')  # TCHW: Time, Channels, Height, Width

# # Initialize video writer (if needed to save output)
# fps = 30  # Change this according to the input video FPS
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# out = cv2.VideoWriter('output_flow.avi', fourcc, fps, (888, 496))

# # Process the video frame by frame
# for i in range(vid.shape[0] - 1):
#     prev_frame = vid[i]
#     curr_frame = vid[i + 1]

#     # Preprocess both frames
#     prev_frame_tensor = preprocess_frame(prev_frame, device)
#     curr_frame_tensor = preprocess_frame(curr_frame, device)
#     # print(prev_frame_tensor.shape) #[1, 3, 496, 888]

#     # Predict optical flow between the previous and current frames
#     with torch.no_grad():
#         flow_list = model(prev_frame_tensor, curr_frame_tensor)

#         # flow_np = []
#         # for flow in flow_list:
#         #     flow_np.append(flow.cpu().numpy()) # Move tensor to CPU before converting to NumPy

#         # print(np.array(flow_np).shape)  # (12, 1, 2, 496, 888)

#         predicted_flow = flow_list[-1]  # Use the final output of the model
#         # print(np.array(predicted_flow.cpu()).shape) # (1, 2, 496, 888)

#     # Convert optical flow to an RGB image for visualizat ion
#     flow_image = flow_to_image(predicted_flow.squeeze(0)).cpu().numpy().transpose(1, 2, 0) #(496, 888, 3)
#     flow_image_bgr = cv2.cvtColor(flow_image.astype(np.uint8), cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV

#     # cv2.imshow('input', flow_image_bgr)
#     # # Wait for 25ms before moving to the next frame
#     # if cv2.waitKey(25) & 0xFF == ord('q'):
#     #     break  # Exit the loop if 'q' is pressed

#     # Save the flow image as part of the output video
#     out.write(flow_image_bgr)

# # Release the video writer
# out.release()
# cv2.destroyAllWindows()

In [6]:
# # Load the video using torchvision
# # dataset_path = Path('H:/Datasets/Celeb_DF/YouTube-real')
# # video_path = "00000.mp4"

# dataset_path = Path('H:/Datasets/Celeb_DF/Celeb-synthesis')
# video_path = "id0_id1_0000.mp4"

# data_path = dataset_path / video_path
# vid, _, _ = read_video(data_path, output_format="TCHW", pts_unit='sec')  # TCHW: Time, Channels, Height, Width

# # Initialize video writer (if needed to save output)
# fps = 30  # Change this according to the input video FPS
# fourcc = cv2.VideoWriter_fourcc(*'MJPG') #h264 MJPG avc1 mpv4
# out = cv2.VideoWriter('output_flow2.avi', fourcc, fps, (888, 496))

# # Parameters
# batch_size = 6  # Number of frames to process in a batch
# num_frames = vid.shape[0]

# # Process the video frame by frame in batches
# for i in range(0, num_frames - 1, batch_size):
#     # Prepare batch of frames
#     batch_prev_frames = []
#     batch_curr_frames = []
    
#     for j in range(batch_size):
#         if i + j < num_frames - 1:
#             batch_prev_frames.append(vid[i + j])
#             batch_curr_frames.append(vid[i + j + 1])

#     # Preprocess batches
#     batch_prev_tensor = torch.cat([preprocess_frame(f, device) for f in batch_prev_frames], dim=0)
#     batch_curr_tensor = torch.cat([preprocess_frame(f, device) for f in batch_curr_frames], dim=0)

#     # Predict optical flow between the previous and current frames
#     with torch.no_grad():
#         # print(batch_prev_tensor.shape)
#         flow_list = model(batch_prev_tensor, batch_curr_tensor)

#         predicted_flows = flow_list[-1]  # Use the final output of the model

#     # Convert optical flows to RGB images for visualization
#     for flow in predicted_flows:
#         flow_image = flow_to_image(flow.squeeze(0)).cpu().numpy().transpose(1, 2, 0)  # (496, 888, 3)
#         flow_image_bgr = cv2.cvtColor(flow_image.astype(np.uint8), cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV

#         # Save the flow image as part of the output video
#         out.write(flow_image_bgr)

# # Release the video writer
# out.release()
# cv2.destroyAllWindows()

In [5]:
# # Custom Dataset class for video frames
# class VideoFrameDataset(Dataset):
#     def __init__(self, video_path):
#         self.video_path = video_path
#         self.vid, _, _ = read_video(video_path, output_format="TCHW", pts_unit='sec')
    
#     def __len__(self):
#         return self.vid.shape[0] - 1  # Last frame has no next frame

#     def __getitem__(self, idx):
#         prev_frame = self.vid[idx]
#         curr_frame = self.vid[idx + 1]
#         return prev_frame, curr_frame
    
# # Preprocessing function (based on RAFT's input expectations)
# def preprocess_frame(frame):
#     frame_tensor = frame.float() / 255.0  # Normalize to [0, 1]
#     frame_resized = resize(frame_tensor, size=[496, 888])  # Resize to match RAFT expectations 500, 892 ->  496, 888
#     return frame_resized.unsqueeze(0)  # Add batch dimension and move to device

# dataset_path = Path('H:/Datasets/Celeb_DF/YouTube-real')
# video_path = "00000.mp4"

# data_path = dataset_path / video_path

# dataset = VideoFrameDataset(data_path)
# data_loader = DataLoader(dataset, batch_size=6, shuffle=False, num_workers=0,pin_memory=True)

# # Initialize video writer (if needed to save output)
# fps = 30  # Change this according to the input video FPS
# fourcc = cv2.VideoWriter_fourcc(*'MJPG') #h264 MJPG avc1 mpv4
# out = cv2.VideoWriter('output_flow2.avi', fourcc, fps, (888, 496))

# # Process the video frames using DataLoader
# for prev_frames, curr_frames in data_loader:
#     # print(prev_frames.shape)
#     # Preprocess batches
#     batch_prev_tensor = torch.stack([preprocess_frame(f, device).to(device) for f in prev_frames]).to(device).squeeze(1)
#     batch_curr_tensor = torch.stack([preprocess_frame(f, device).to(device) for f in curr_frames]).to(device).squeeze(1)

#     with torch.no_grad():
#         # print(batch_prev_tensor.shape)
#         flow_list = model(batch_prev_tensor, batch_curr_tensor)
#         predicted_flows = flow_list[-1]  # Use the final output of the model

#     # Convert optical flows to RGB images for visualization
#     for flow in predicted_flows:
#         flow_image = flow_to_image(flow.squeeze(0)).cpu().numpy().transpose(1, 2, 0)  # (496, 888, 3)
#         flow_image_bgr = cv2.cvtColor(flow_image.astype(np.uint8), cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV
#         out.write(flow_image_bgr)

# # Release the video writer
# out.release()
# cv2.destroyAllWindows()

In [10]:
# def compress_video(input_file, output_file, target_bitrate=500000):
#     # Open the input file
#     input_container = av.open(input_file)
    
#     # Create an output container to write the compressed video
#     output_container = av.open(output_file, mode='w')

#     # Set codec for the output video stream (e.g., H.264)
#     output_stream = output_container.add_stream('h264', rate=30)  # 30 fps

#     # Set the bitrate for compression
#     output_stream.bit_rate = target_bitrate  # Bitrate in bits per second

#     for frame in input_container.decode(video=0):
#         # Re-encode the frames to the output video stream
#         packet = output_stream.encode(frame)
#         if packet:
#             output_container.mux(packet)

#     # Flush any remaining packets
#     output_container.mux(output_stream.encode())
    
#     # Close the containers
#     input_container.close()
#     output_container.close()

#     print(f"Video compressed successfully to: {output_file}")

# # Example usage
# input_video = 'output_flow.avi'
# output_video = 'output_flow_compressed.mp4'

# compress_video(input_video, output_video, target_bitrate=1500000)

Video compressed successfully to: output_flow_compressed3.mp4


In [19]:
# # Custom Dataset class for multiple videos
# class MultiVideoFrameDataset(Dataset):
#     def __init__(self, video_dir):
#         self.video_paths = list(video_dir.glob("*.mp4"))  # Assume all videos are .mp4 files
#         self.video_frames = []
#         self.frame_indices = []

#         # Preload all video frames and keep track of frame indices for all videos
#         for video_path in self.video_paths:
#             try:
#                 vid, _, _ = read_video(video_path, output_format="TCHW", pts_unit='sec')
#                 for i in range(len(vid) - 1):  # Loop over frames in each video
#                     self.video_frames.append((video_path, vid[i], vid[i + 1]))  # (video, frame_i, frame_i+1)
#                     self.frame_indices.append(i)
#             except Exception as e:
#                 print(f"Error reading video file: {video_path}")
#                 print(e)

#     def __len__(self):
#         return len(self.video_frames)

#     def __getitem__(self, idx):
#         video_path, prev_frame, curr_frame = self.video_frames[idx]
#         return video_path, prev_frame, curr_frame

# # Custom collate_fn to handle video paths and frames separately
# def custom_collate(batch):
#     video_paths, prev_frames, curr_frames = zip(*batch)  # Unzip the batch into paths, previous frames, and current frames
#     return list(video_paths), torch.stack(prev_frames), torch.stack(curr_frames)

# # Preprocessing function (based on RAFT's input expectations)
# def preprocess_frame(frame):
#     frame_tensor = frame.float() / 255.0  # Normalize to [0, 1]
#     frame_resized = resize(frame_tensor, size=[496, 888])  # Resize to match RAFT expectations 500, 892 ->  496, 888
#     return frame_resized # Change to [C, H, W]  # Add batch dimension and move to device

# def compress_video(input_file, output_file, target_bitrate=500000):
#     # Open the input file
#     input_container = av.open(input_file)
    
#     # Create an output container to write the compressed video
#     output_container = av.open(output_file, mode='w')

#     # Set codec for the output video stream (e.g., H.264)
#     output_stream = output_container.add_stream('h264', rate=30)  # 30 fps

#     # Set the bitrate for compression
#     output_stream.bit_rate = target_bitrate  # Bitrate in bits per second

#     for frame in input_container.decode(video=0):
#         # Re-encode the frames to the output video stream
#         packet = output_stream.encode(frame)
#         if packet:
#             output_container.mux(packet)

#     # Flush any remaining packets
#     output_container.mux(output_stream.encode())
    
#     # Close the containers
#     input_container.close()
#     output_container.close()

#     print(f"Video compressed successfully to: {output_file}")

# # Main function to process videos
# def process_videos(video_dir, batch_size=6, num_workers=0, output_dir="output_videos"):
#     dataset = MultiVideoFrameDataset(video_dir)
#     data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True,collate_fn=custom_collate)

#     # Ensure output directory exists
#     Path(output_dir).mkdir(parents=True, exist_ok=True)

#     # Dictionary to store video writers for different videos
#     video_writers = {}
#     cur_vid = 'start'

#     for video_path, prev_frames, curr_frames in data_loader:
#         # Preprocess batches
#         batch_prev_tensor = torch.stack([preprocess_frame(f).to(device) for f in prev_frames]).to(device)
#         batch_curr_tensor = torch.stack([preprocess_frame(f).to(device) for f in curr_frames]).to(device)

#         with torch.no_grad():
#             # print(batch_prev_tensor.shape)
#             # print(batch_curr_tensor.shape)
#             flow_list = model(batch_prev_tensor, batch_curr_tensor)
#             predicted_flows = flow_list[-1]  # Use the final output of the model

#         # Loop through each batch and write output to corresponding video
#         for idx, flow in enumerate(predicted_flows):
#             # Get the video path and ensure the video writer is initialized
#             video_file = video_path[idx]
#             # print(video_file)
#             if cur_vid == 'start':
#                 cur_vid = video_file
#                 # print(cur_vid)

#             if video_file != cur_vid:
#                 output_video_path = Path(output_dir) / f"{cur_vid.stem}_flow.mp4"
#                 print(output_video_path)
#                 compress_video(temp_path, output_video_path, target_bitrate=1500000)
#                 cur_vid = video_file

#             if video_file not in video_writers:
#                 temp_path = "temp.avi"
#                 fps = 30  # Change according to input video FPS
#                 fourcc = cv2.VideoWriter_fourcc(*'MJPG')
#                 video_writers[video_file] = cv2.VideoWriter(temp_path, fourcc, fps, (888, 496))

#             # Convert flow to image and write to video
#             flow_image = flow_to_image(flow.squeeze(0)).cpu().numpy().transpose(1, 2, 0)  # (496, 888, 3)
#             flow_image_bgr = cv2.cvtColor(flow_image.astype(np.uint8), cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV
#             video_writers[video_file].write(flow_image_bgr)


#     # Release all video writers
#     for writer in video_writers.values():
#         writer.release()

#     cv2.destroyAllWindows()

# # Path to directory containing videos
# # video_dir = Path('H:/Datasets/Celeb_DF/YouTube-real')
# video_dir = Path('H:/Datasets/Test')
# # Output directory to save flow videos
# output_dir = "output_flow_videos"

# # Process videos
# process_videos(video_dir, batch_size=6, num_workers=0, output_dir=output_dir)

H:\Datasets\Test\00000.mp4
output_flow_videos\00000_flow.mp4
Video compressed successfully to: output_flow_videos\00000_flow.mp4
output_flow_videos\00001_flow.mp4
Video compressed successfully to: output_flow_videos\00001_flow.mp4


In [3]:
# Custom Dataset class for video frames
class VideoFrameDataset(Dataset):
    def __init__(self, video_path, transform):
        self.video_path = video_path
        self.vid, _, _ = read_video(video_path, output_format="TCHW", pts_unit='sec')

        # Clip frames to the fixed number `max_frames`
        self.max_frames = 300
        self.vid = self._clip_frames(self.vid)

        # sample n number of frames from clipped video
        self.n_frames = 150
        self.vid = self._sample_frames(self.vid)

        self.vid = transform(self.vid)
        # print(self.vid.shape)
    
    def __len__(self):
        return self.vid.shape[0] - 1  # Last frame has no next frame

    def __getitem__(self, idx):
        prev_frame = self.vid[idx]
        curr_frame = self.vid[idx + 1]
        return prev_frame, curr_frame
    
    def _clip_frames(self, frames):
        """Clip the video frames to a fixed number of frames `max_frames`."""
        # If the video has more frames than max_frames, clip it to the first `max_frames`
        if len(frames) > self.max_frames:
            return frames[:self.max_frames]
        
        # If the video has fewer frames than `max_frames`, pad with zeros
        pad_size = self.max_frames - len(frames)
        pad_frames = torch.zeros((pad_size, *frames.shape[1:]))  # Create padding frames with same size
        frames = torch.cat([frames, pad_frames], dim=0)
        
        return frames
    
    def _sample_frames(self, frames):
        """Sample n number of frames from cliped video"""
        if len(frames) > self.n_frames:
            indices = torch.linspace(0, len(frames)-1, steps=self.n_frames).long()
            frames = frames[indices]  
        
        return frames


# Preprocessing function (based on RAFT's input expectations)
# def preprocess_frame(frame):
#     frame_tensor = frame.float() / 255.0  # Normalize to [0, 1]
#     frame_resized = resize(frame_tensor, size=[496, 888])  # Resize to match RAFT expectations 500, 892 ->  496, 888
#     return frame_resized.unsqueeze(0)  # Add batch dimension and move to device

preprocess = T.Compose([
    T.ConvertImageDtype(torch.float32),
    T.Normalize(mean=(0.43216, 0.394666, 0.37645),
                std=(0.22803, 0.22145, 0.216989)),
    T.Resize([256, 256]),
])

# def preprocess_frame(frame):
#     return preprocess(frame).unsqueeze(0)  # Preprocess and add batch dimension

def process_single_video(video_path, model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load the dataset
    dataset = VideoFrameDataset(video_path,preprocess)
    data_loader = DataLoader(dataset, batch_size=75, shuffle=False, num_workers=0, pin_memory=True)

    # Initialize video writer
    fps = 15  # Adjust if needed
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    # output_file = output_dir / (video_path.stem + '_flow.avi')
    output_file = "temp.avi"
    out = cv2.VideoWriter(output_file, fourcc, fps, (256, 256))

    # Process video frames
    for prev_frames, curr_frames in data_loader:
        # Preprocess batches
        batch_prev_tensor = torch.stack([f.unsqueeze(0).to(device) for f in prev_frames]).squeeze(1)
        batch_curr_tensor = torch.stack([f.unsqueeze(0).to(device) for f in curr_frames]).squeeze(1)
        # print(batch_prev_tensor.shape)

        with torch.no_grad(): 
            with torch.autocast(device_type="cuda"):
                # Predict optical flows
                flow_list = model(batch_prev_tensor, batch_curr_tensor)
                predicted_flows = flow_list[-1]  # Final output

        # Convert optical flows to RGB and write to video
        for flow in predicted_flows:
            flow_image = flow_to_image(flow.squeeze(0)).cpu().numpy().transpose(1, 2, 0)  # (496, 888, 3)
            flow_image_bgr = cv2.cvtColor(flow_image.astype(np.uint8), cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV
            out.write(flow_image_bgr)

        # # Clear tensors and free memory
        # del batch_prev_tensor, batch_curr_tensor, flow_list, predicted_flows
        # torch.cuda.empty_cache()

    # Release resources
    out.release()

def compress_video(input_file, output_file, target_bitrate=500000):
    # Open the input file
    input_container = av.open(input_file)
    
    # Create an output container to write the compressed video
    output_container = av.open(output_file, mode='w')

    # Set codec for the output video stream (e.g., H.264)
    output_stream = output_container.add_stream('h264', rate=15)  # 30 fps

    # Set the bitrate for compression
    output_stream.bit_rate = target_bitrate  # Bitrate in bits per second

    # Set the resolution of the output video stream based on the input stream
    output_stream.width = 256
    output_stream.height = 256

    for frame in input_container.decode(video=0):
        # Re-encode the frames to the output video stream
        packet = output_stream.encode(frame)
        if packet:
            output_container.mux(packet)

    # Flush any remaining packets
    output_container.mux(output_stream.encode())
    
    # Close the containers
    input_container.close()
    output_container.close()

    # print(f"Video compressed successfully to: {output_file}")

def process_videos_in_folder(folder_path, model, output_dir):
    video_paths = sorted(Path(folder_path).glob('*.mp4'))  # Assumes .mp4, adjust if necessary
    output_dir.mkdir(parents=True, exist_ok=True)

    for video_path in video_paths:
        # print(f"Processing video: {video_path}")
        process_single_video(video_path, model)

        # Optionally compress the video after processing
        input_vid = "temp.avi"
        compressed_output_file = output_dir / (video_path.stem + '.mp4')
        compress_video(input_vid, compressed_output_file, target_bitrate=1500000)


# folder_path = Path('H:/Datasets/maliciousai/data/Celeb-DF-v2/YouTube-real')
# output_dir = Path('H:/Datasets/maliciousai/data/Celeb-DF-v2-Flow/YouTube-real')

# folder_path = Path('H:/Datasets/maliciousai/data/Celeb-DF-v2/Celeb-real')
# output_dir = Path('H:/Datasets/maliciousai/data/Celeb-DF-v2-Flow/Celeb-real')

folder_path = Path('H:/Datasets/maliciousai/data/Celeb-synthesis')
output_dir = Path('H:/Datasets/maliciousai/data/Celeb-DF-v2-Flow/Celeb-synthesis')

process_videos_in_folder(folder_path, model, output_dir) # 3.8s per video